<a href="https://colab.research.google.com/github/usma11dia0/training_of_kaggle/blob/main/google_colab/100_knock/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9100%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF_%E7%B7%B4%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

##解答リンク先
https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/blob/master/docker/work/answer/ans_preprocess_knock_Python.ipynb

In [ ]:
# pipでオリジナルの解答に必要なライブラリーをインストール
!pip install --upgrade pip
!pip install -U pandas numpy scikit-learn imbalanced-learn

# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_category = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')
df_store = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv')
df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt.loc[0:9, ["sales_ymd","customer_id","product_cd","amount"]]

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
tmp = df_receipt.loc[0:10, ["sales_ymd","customer_id","product_cd","amount"]]
tmp.head(3)
tmp.rename(columns={"sales_ymd":"sales_date"})

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
tmp = df_receipt.loc[:, ["sales_ymd","customer_id","product_cd","amount"]]
tmp[tmp["customer_id"]=="CS018205000001"]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
tmp = df_receipt.loc[:, ["sales_ymd","customer_id","product_cd","amount"]]
cond = (tmp["customer_id"]=="CS018205000001") & (tmp["amount"]>=1000)
tmp[cond]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
tmp = df_receipt.loc[:, ["sales_ymd","customer_id","product_cd","quantity","amount"]]
cond = (tmp["customer_id"]=="CS018205000001") & ((tmp["amount"]>=1000) | (tmp["quantity"]>= 5))
tmp[cond]

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
tmp = df_receipt.loc[:, ["sales_ymd","customer_id","product_cd","amount"]]
cond = (tmp["customer_id"]=="CS018205000001") & ((tmp["amount"]>=1000) & (tmp["amount"] <= 2000))
tmp[cond]

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
tmp = df_receipt.loc[:, ["sales_ymd","customer_id","product_cd","amount"]]
cond = (tmp["customer_id"]=="CS018205000001") & (tmp["product_cd"] != "P071401019")
tmp[cond]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
23,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.5966,35.63614,883.0
24,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.7186,35.76686,801.0
25,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.7436,35.60981,845.0
29,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.4235,35.63787,810.0
30,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.6513,35.70273,841.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0
50,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.8215,35.67898,814.0


In [ ]:
df_store.query('not(prefecture_cd == "13") and not(floor_area > 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
23,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.5966,35.63614,883.0
24,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.7186,35.76686,801.0
25,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.7436,35.60981,845.0
29,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.4235,35.63787,810.0
30,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.6513,35.70273,841.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0
50,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.8215,35.67898,814.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
cond = df_store["store_cd"].str.contains("^S14")
df_store[cond].head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
cond = df_customer['customer_id'].str.contains("1$")
df_customer[cond].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
cond = df_store["address"].str.contains("^.*横浜市.*$")
df_store[cond]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
cond = df_customer["status_cd"].str.contains("^[A-F]")
df_customer[cond].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
cond = df_customer["status_cd"].str.contains("[1-9]$")
df_customer[cond].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
cond = df_customer["status_cd"].str.contains("^[A-F].*[1-9]$")
df_customer[cond].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
cond = df_store["tel_no"].str.contains("[0-9]{3}-[0-9]{3}-[0-9]{4}")
df_store[cond].head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_receipt['amount'].rank(method="min", ascending=False)

0          55059.0
1          95294.0
2          51700.0
3         104339.0
4          89871.0
            ...   
104676    103127.0
104677     38644.0
104678     52588.0
104679     57032.0
104680     62707.0
Name: amount, Length: 104681, dtype: float64

In [ ]:
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']],df_receipt['amount'].rank(method="min", ascending=False)],axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking').head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_receipt['amount'].rank(method="first", ascending=False)
df_tmp = pd.concat([df_receipt[['customer_id','amount']],df_receipt['amount'].rank(method="first", ascending=False)],axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking').head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt['customer_id'].unique())

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({"amount":"sum","quantity":"sum"}).reset_index().head()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403


In [ ]:
# reset_index()を付けなかった場合
df_receipt.groupby('store_cd').agg({"amount":"sum","quantity":"sum"}).head()

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'max'}).reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
#顧客ID(customer_id)でグループ化
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.head()

customer_id sales_ymd          
                        max       min
0  CS001113000004  20190308  20190308
1  CS001114000005  20190731  20180503
2  CS001115000010  20190405  20171228
3  CS001205000004  20190625  20170914
4  CS001205000006  20190224  20180207

In [ ]:
#カラム構造確認 ※インデックス更新はリスト化の必要あり
df_tmp.columns

MultiIndex([('customer_id',    ''),
            (  'sales_ymd', 'max'),
            (  'sales_ymd', 'min')],
           )

In [ ]:
#カラム名の抽出確認
df_tmp.columns[1]

('sales_ymd', 'max')

In [ ]:
#MultiIndex 1以降のカラム名結合 (リスト化)
new_columns = ["_".join(pair) for pair in df_tmp.columns[1:]]
new_columns

['sales_ymd_max', 'sales_ymd_min']

In [ ]:
#上記で作成したnew_columnsの0番目に, MultiIndexの'customer_id'を挿入
new_columns.insert(0, df_tmp.columns[0][0])
new_columns

['customer_id', 'sales_ymd_max', 'sales_ymd_min']

In [ ]:
#グループ化したDFのカラム名を新しいnew_columnsへ書き換え
df_tmp.columns = new_columns
df_tmp

,customer_id,sales_ymd_max,sales_ymd_min
0,CS001113000004,20190308,20190308
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
...,...,...,...
8302,CS051513000004,20190719,20190719
8303,CS051515000002,20191025,20191025
8304,CS052212000002,20191017,20191017
8305,CS052514000001,20190822,20190822


In [ ]:
#最も新しい売上日と古い売上日で両者が異なるデータを抽出
df_tmp.query('sales_ymd_max != sales_ymd_min').head(10)

,customer_id,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby("store_cd").agg({'amount':'median'}).reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
# groupbyには最頻値(mode)が無い。※以下は出来ない。
# df_receipt.groupby("store_cd").agg({'product_cd':'mode'}).reset_index().sort_values('amount',ascending=False).head(5)

#理由
#mode()は返り値にscalerではなくSeriesを返すため。

#例 [りんご、りんご、みかん、みかん、なし]
#→最頻値は(りんご、みかん)で一意に値(scaler)としてに定まらない。

In [ ]:
#ラムダ式(無名関数)へ最頻値を出力する式を記載し、aggメソッドの引数に渡す
#groupby以降の[[]]でDB内へ格納するカラムを指定
df_receipt.groupby("store_cd")[["product_cd"]].agg(lambda x : x.mode()).reset_index().head(10)

,store_cd,product_cd
0,S12007,P060303001
1,S12013,P060303001
2,S12014,P060303001
3,S12029,P060303001
4,S12030,P060303001
5,S13001,P060303001
6,S13002,P060303001
7,S13003,P071401001
8,S13004,P060303001
9,S13005,P040503001


In [ ]:
#別解　ラムダ式を用いずに外部で定義した関数を用いる場合
def dataframe_mode(x):
  return x.mode()
df_receipt.groupby("store_cd")[["product_cd"]].agg(dataframe_mode).reset_index().head(10)

,store_cd,product_cd
0,S12007,P060303001
1,S12013,P060303001
2,S12014,P060303001
3,S12029,P060303001
4,S12030,P060303001
5,S13001,P060303001
6,S13002,P060303001
7,S13003,P071401001
8,S13004,P060303001
9,S13005,P040503001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd')[['amount']].var(ddof=0).reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby("store_cd")[['amount']].std(ddof=0).reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
#df[]とdf[[]]の違い
#df[]だと出力がSeriesになる。df[[]]だとDFとなる。
df_receipt['amount'].quantile([.25,.50,.75,1])

0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

In [ ]:
df_receipt[['amount']].quantile(np.arange(1,5)/4)

,amount
0.25,102.0
0.50,170.0
0.75,288.0
1.00,10925.0


---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_receipt.groupby('store_cd')[['amount']].mean().reset_index().query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_receipt[cond].groupby('customer_id').agg({'amount':'sum'}).mean()

amount    2547.742235
dtype: float64

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
#顧客IDが"Z"から始まるものをFalseとしたBool型のSeriesを作成
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)

#Seriesを適用させて顧客IDごとに売上金額を合計してグループ化、
tmp_df = df_receipt[cond].groupby("customer_id")[['amount']].sum()

#全顧客の平均を求める
amount_mean = tmp_df.mean()

#amount_meanは値一つのSeries型であるため、抽出してint型へ変換
amount_mean_value = amount_mean[0]

#group化対象の項目もカラムへ含む
tmp_df = tmp_df.reset_index()

#平均以上の売上金額である顧客を抽出
tmp_df.query('amount >= @amount_mean_value').head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [ ]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [ ]:
#両者のDFで"store_id"が共通しているカラムであるため、"store_id"をキーとして内部結合
#mergeの第一引数には結合するDF(キーと結合したいカラム)を指定する。
df_receipt.merge(df_store[['store_cd','store_name']],how="inner",on="store_cd").head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
df_product.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0


In [ ]:
df_category.head(3)

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類


In [ ]:
df_product.merge(df_category[['category_major_cd','category_small_name']], how="inner", on="category_major_cd").query('category_small_name == "弁当類"').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
22,P040101002,4,401,40101,218.0,164.0,弁当類
44,P040101003,4,401,40101,230.0,173.0,弁当類
66,P040101004,4,401,40101,248.0,186.0,弁当類
88,P040101005,4,401,40101,268.0,201.0,弁当類
110,P040101006,4,401,40101,298.0,224.0,弁当類
132,P040101007,4,401,40101,338.0,254.0,弁当類
154,P040101008,4,401,40101,420.0,315.0,弁当類
176,P040101009,4,401,40101,498.0,374.0,弁当類
198,P040101010,4,401,40101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [ ]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [ ]:
#.df_customerから非会員を除外
cond = df_customer['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_customer[cond]

#df_tmp1より女性のみ抽出
df_tmp2 = df_tmp.query('gender_cd == 1')
df_tmp2

#外部結合(LEFT JOIN)の実施 (left = df_tmp【メイン】,　right = df_receipt)
df_tmp3 = df_tmp2.merge(df_receipt[['customer_id','amount']], how="left", on="customer_id")
df_tmp3

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,100.0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,320.0
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,2400.0
4,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,448.0
...,...,...,...,...,...,...,...,...,...,...,...,...
70775,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,390.0
70776,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,88.0
70777,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,2200.0
70778,CS033512000184,池谷 華子,1,女性,1964-06-05,54,245-0016,神奈川県横浜市泉区和泉町**********,S14033,20160206,0-00000000-0,NaN


In [ ]:
#結合後のDFにて売上合計でグループ化。買い物の実績がない顧客については"0"を代入。
df_tmp3.groupby('customer_id')[['amount']].sum().fillna(0).reset_index().sort_values('amount', ascending=False)

,customer_id,amount
9229,CS017415000097,23086.0
8475,CS015415000185,20153.0
14383,CS031414000051,19202.0
13265,CS028415000007,19127.0
6468,CS010214000010,18585.0
...,...,...
6904,CS011315000036,0.0
6906,CS011315000065,0.0
6907,CS011315000075,0.0
6908,CS011315000077,0.0


In [ ]:
#別解
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum
df_tmp = df_customer.query('gender_cd == 1 and not customer_id.str.startswith("Z")',engine='python')
pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).sort_values('amount',ascending=False)

,customer_id,amount
13809,CS017415000097,23086.0
10356,CS015415000185,20153.0
11051,CS031414000051,19202.0
3920,CS028415000007,19127.0
12043,CS010214000010,18585.0
...,...,...
7685,CS028714000027,0.0
7686,CS008611000032,0.0
7688,CS022313000023,0.0
7693,CS047312000002,0.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
#非会員(顧客IDがZから始まるもの)を除外
cond = df_receipt['customer_id'].str.contains('^[^Z]', regex=True)
df_tmp1 = df_receipt[cond]

#売上日数の多い顧客上位20名を抽出
#同一日に記録された複数レコードは削除　※同一日付を売上日数として複数カウントしてしまうため。
#subsetを指定する事で、両者二つが共に重複しているもののみを削除
df_tmp2 = df_tmp1.drop_duplicates(subset=["customer_id","sales_ymd"], keep="first")

#重複削除後のDFにて売り上げ日数でグループ化
df_tmp3 = df_tmp2.groupby('customer_id')[['sales_ymd']].count().sort_values('sales_ymd',ascending=False).head(20)

# 売上金額合計の多い顧客上位20名を抽出
df_tmp4 = df_tmp1.groupby('customer_id')[['amount']].sum().sort_values('amount',ascending=False).head(20)

#完全外部結合を実施
df_tmp5 = df_tmp3.merge(df_tmp4, how="outer", on="customer_id")
df_tmp5.head()

,sales_ymd,amount
customer_id,,
CS040214000008,23.0,NaN
CS015415000185,22.0,20153.0
CS010214000010,22.0,18585.0
CS010214000002,21.0,NaN
CS028415000007,21.0,19127.0


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [ ]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [ ]:
len(df_product)*len(df_store)

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
#df_receiptにて売上日付でグループ化し、売上金額を合計
df_tmp = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum()
df_tmp1 = df_tmp.reset_index()

#df_tmp1にて1行前との差分を導出
df_tmp2 = df_tmp1.diff(periods=1)[['amount']]

#dfを結合。カラム名も変更
df_tmp3 = pd.concat([df_tmp1,df_tmp2],axis=1)
df_tmp3.columns = ["sales_ymd",	"amount",	"lag_amount"]
df_tmp3.head()

,sales_ymd,amount,lag_amount
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0


In [ ]:
#別解  df.diffを用いずに実装する方法

#df_receiptにて売上日付でグループ化し、売上金額を合計
df_ans = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
#df_ansを一行下へシフト
df_ans.shift()

,sales_ymd,amount
0,NaN,NaN
1,20170101.0,33723.0
2,20170102.0,24165.0
3,20170103.0,27503.0
4,20170104.0,36165.0
...,...,...
1029,20191026.0,51771.0
1030,20191027.0,37484.0
1031,20191028.0,40161.0
1032,20191029.0,36091.0


In [ ]:
#一行下へシフトさせたDFと元のDFを結合
df_tmp = pd.concat([df_ans,df_ans.shift()],axis=1)
df_tmp
df_tmp.columns = ['sales_ymd', 'amount', 'lag_ymd', 'lag_amount']
df_tmp.head()

,sales_ymd,amount,lag_ymd,lag_amount
0,20170101,33723,NaN,NaN
1,20170102,24165,20170101.0,33723.0
2,20170103,27503,20170102.0,24165.0
3,20170104,36165,20170103.0,27503.0
4,20170105,37830,20170104.0,36165.0


In [ ]:
#カラムより売上合計の差分を導出し追加。
df_tmp['diff_amount'] = df_tmp['amount'] - df_tmp['lag_amount']
df_tmp.head()

,sales_ymd,amount,lag_ymd,lag_amount,diff_amount
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
df_sales_amount_by_date = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
df_sales_amount_by_date

,sales_ymd,amount
0,20170101,33723
1,20170102,24165
2,20170103,27503
3,20170104,36165
4,20170105,37830
...,...,...
1029,20191027,37484
1030,20191028,40161
1031,20191029,36091
1032,20191030,26602


In [ ]:
df_sales_amount_by_date.shift()

,sales_ymd,amount
0,NaN,NaN
1,20170101.0,33723.0
2,20170102.0,24165.0
3,20170103.0,27503.0
4,20170104.0,36165.0
...,...,...
1029,20191026.0,51771.0
1030,20191027.0,37484.0
1031,20191028.0,40161.0
1032,20191029.0,36091.0


In [ ]:
#縦に付ける

# # 1日前
# df_tmp = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift()],axis=1)
# df_tmp.columns = ['sales_ymd', 'amount', 'lag_sales_ymd', 'lag_amount']
# df_tmp

# # 2日前
# df_tmp2 = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(2)],axis=1)
# df_tmp2.columns = ['sales_ymd', 'amount', 'lag_sales_ymd', 'lag_amount']
# df_tmp2
# df_tmp2

# pd.concat([df_tmp,df_tmp2],axis=0).sort_values('sales_ymd').head(10)

for i in range(1,4):
    df_tmp = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(i)],axis=1)
    df_tmp.columns = ['sales_ymd', 'amount', 'lag_sales_ymd', 'lag_amount']
    if i == 1:
      df_base = df_tmp
    else:
      df_base = pd.concat([df_base, df_tmp],axis=0)

# cond = df_base.notnull()
# cond
df_base.dropna().sort_values(['sales_ymd','lag_sales_ymd']).head(10)

,sales_ymd,amount,lag_sales_ymd,lag_amount
1,20170102,24165,20170101.0,33723.0
2,20170103,27503,20170101.0,33723.0
2,20170103,27503,20170102.0,24165.0
3,20170104,36165,20170101.0,33723.0
3,20170104,36165,20170102.0,24165.0
3,20170104,36165,20170103.0,27503.0
4,20170105,37830,20170102.0,24165.0
4,20170105,37830,20170103.0,27503.0
4,20170105,37830,20170104.0,36165.0
5,20170106,32387,20170103.0,27503.0


In [ ]:
# #横に付ける

# #1日前
# df_tmp1 = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift()],axis=1)
# df_tmp1.columns = ['sales_ymd', 'amount', 'lag1_sales_ymd', 'lag1_amount']
# df_tmp1

# #2日前
# df_tmp2 = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(2)],axis=1)
# df_tmp2.columns = ['sales_ymd', 'amount', 'lag2_sales_ymd', 'lag2_amount']
# df_tmp2
# df_tmp2 = df_tmp2.iloc[:,-2:]

# pd.concat([df_tmp1,df_tmp2],axis=1)

for i in range(1,4):
  df_tmp = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(i)], axis=1)
  df_tmp.columns = ['sales_ymd', 'amount', f'lag{i}_sales_ymd', f'lag{i}_amount']

  if i == 1:
    df_base = df_tmp
  else:
    df_tmp = df_tmp.iloc[:,-2:]
    df_base = pd.concat([df_base,df_tmp], axis=1)

df_base.dropna().sort_values('sales_ymd')

,sales_ymd,amount,lag1_sales_ymd,lag1_amount,lag2_sales_ymd,lag2_amount,lag3_sales_ymd,lag3_amount
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
...,...,...,...,...,...,...,...,...
1029,20191027,37484,20191026.0,51771.0,20191025.0,28833.0,20191024.0,31868.0
1030,20191028,40161,20191027.0,37484.0,20191026.0,51771.0,20191025.0,28833.0
1031,20191029,36091,20191028.0,40161.0,20191027.0,37484.0,20191026.0,51771.0
1032,20191030,26602,20191029.0,36091.0,20191028.0,40161.0,20191027.0,37484.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [ ]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [ ]:
df_tmp = df_customer.merge(df_receipt, how="inner", on="customer_id")
df_tmp['age_2'] = df_tmp['age'].apply(lambda x : math.floor(x/10)*10)
df_tmp

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount,age_2
0,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20170507,1494115200,S13031,1102,1,P060103001,1,100,40
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20171026,1508976000,S13031,1182,1,P090203004,1,320,40
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20190325,1553472000,S13031,1192,1,P071401025,1,2400,40
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20170111,1484092800,S13031,1132,2,P071203007,1,448,40
4,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20190325,1553472000,S13031,1192,2,P070805011,1,258,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65677,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,20191028,1572220800,S12029,1182,1,P060102002,1,88,40
65678,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,20190806,1565049600,S12029,1132,2,P060101007,1,180,40
65679,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,20180611,1528675200,S12029,1162,2,P090204049,1,390,40
65680,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,20180305,1520208000,S12029,1132,2,P050602001,1,88,40


In [ ]:
table = pd.pivot_table(df_tmp, values="amount", index=['age_2'], columns=['gender_cd'], aggfunc=np.sum).reset_index()
table

gender_cd,age_2,0,1,9
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
df_tmp[['age_2','gender','amount']]

,age_2,gender,amount
0,40,女性,100
1,40,女性,320
2,40,女性,2400
3,40,女性,448
4,40,女性,258
...,...,...,...
65677,40,女性,88
65678,40,女性,180
65679,40,女性,390
65680,40,女性,88


In [ ]:
table_2 = table.set_index('age_2').stack().reset_index().replace({0:"男性",1:"女性",9:"不明"})
table_2.columns = ["age_2", "gender_cd", "amount"]
table_2.head()

,age_2,gender_cd,amount
0,10,男性,1591.0
1,10,女性,149836.0
2,10,不明,4317.0
3,20,男性,72940.0
4,20,女性,1363724.0


In [ ]:
#.stack()を付けない場合
table_3 = table.set_index('age_2').reset_index().replace({0:"男性",1:"女性",9:"不明"})
table_3.head()

gender_cd,age_2,0,1,9
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_customer['birth_day'].head()

0    1981-04-29
1    1952-04-01
2    1976-10-04
3    1933-03-27
4    1995-03-29
Name: birth_day, dtype: object

In [ ]:
pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%d%m')
tmp = pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['birth_day'])],axis=1).head()
tmp

,customer_id,birth_day
0,CS021313000114,1981-04-29
1,CS037613000071,1952-04-01
2,CS031415000172,1976-10-04
3,CS028811000001,1933-03-27
4,CS001215000145,1995-03-29


In [ ]:
#参考　変換後のデータ型
tmp['birth_day'].head()

0   1981-04-29
1   1952-04-01
2   1976-10-04
3   1933-03-27
4   1995-03-29
Name: birth_day, dtype: datetime64[ns]

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
time_series = pd.to_datetime(df_customer['application_date'].astype(str))
pd.concat([df_customer['customer_id'],time_series], axis=1)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
...,...,...
21966,CS002512000474,2017-11-10
21967,CS029414000065,2015-03-13
21968,CS012403000043,2015-04-06
21969,CS033512000184,2016-02-06


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
time_series = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
pd.concat([df_receipt['receipt_no'],df_receipt['receipt_sub_no'],time_series],axis=1).head()

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
time_series_2 = pd.to_datetime(df_receipt['sales_epoch'],unit='s')
pd.concat([df_receipt['receipt_no'],df_receipt['receipt_sub_no'],time_series_2], axis=1).head()

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
time_series_3 = pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%Y')
pd.concat([df_receipt['receipt_no'],df_receipt['receipt_sub_no'],time_series_3], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
...,...,...,...
104676,1132,2,2018
104677,1132,2,2019
104678,1122,1,2017
104679,1142,1,2017


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
time_series_4 = pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%m')
pd.concat([df_receipt['receipt_no'],df_receipt['receipt_sub_no'],time_series_4], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
...,...,...,...
104676,1132,2,02
104677,1132,2,09
104678,1122,1,03
104679,1142,1,03


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
time_series_4 = pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%d')
pd.concat([df_receipt['receipt_no'],df_receipt['receipt_sub_no'],time_series_4], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
...,...,...,...
104676,1132,2,21
104677,1132,2,11
104678,1122,1,11
104679,1142,1,31


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
cond  = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]
df_tmp2 = df_tmp.groupby('customer_id')[['amount']].sum().reset_index()

def amount_judge(x):
  if x > 2000:
    return 1
  else:
    return 0

df_tmp2['flag'] = df_tmp2['amount'].apply(lambda x : 1 if x>2000 else 0)
df_tmp2

,customer_id,amount,flag
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
...,...,...,...
8301,CS051212000001,336,0
8302,CS051513000004,551,0
8303,CS051515000002,265,0
8304,CS052212000002,192,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
int(df_customer['postal_cd'][1][0:3])


136

In [ ]:
def make_flag(postal_cd):
  if 100 <= int(postal_cd[0:3]) < 210:
    return 1
  else:
    return 0

df_customer['postal_cd_flag'] = df_customer['postal_cd'].apply(make_flag)
df_customer

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,postal_cd_flag
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,1
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,1
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
21966,CS002512000474,市村 夏希,1,女性,1959-10-12,59,185-0034,東京都国分寺市光町**********,S13002,20171110,0-00000000-0,1
21967,CS029414000065,上村 怜奈,1,女性,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F,0
21968,CS012403000043,堀越 明,0,男性,1972-12-16,46,231-0825,神奈川県横浜市中区本牧間門**********,S14012,20150406,0-00000000-0,0
21969,CS033512000184,池谷 華子,1,女性,1964-06-05,54,245-0016,神奈川県横浜市泉区和泉町**********,S14033,20160206,0-00000000-0,0


In [ ]:
df_tmp = df_customer.merge(df_receipt[['customer_id','amount']],how='inner', on="customer_id")
df_tmp = df_tmp.drop_duplicates(['customer_id'], keep='first')
df_tmp
df_tmp.groupby('postal_cd_flag')[['customer_id']].count()

,customer_id
postal_cd_flag,
0,3906
1,4400


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
def slice_pref(x):
  return x[0:3]

series_pref_code = df_customer['address'].apply(lambda x : x[0:3]).replace({"埼玉県":11, "千葉県":12, "東京都":13, "神奈川":14})

df_tmp = pd.concat([df_customer[['customer_id','address']],series_pref_code],axis=1)
df_tmp.columns = ['customer_id', 'address', 'address_code']
df_tmp

,customer_id,address,address_code
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
...,...,...,...
21966,CS002512000474,東京都国分寺市光町**********,13
21967,CS029414000065,千葉県浦安市富士見**********,12
21968,CS012403000043,神奈川県横浜市中区本牧間門**********,14
21969,CS033512000184,神奈川県横浜市泉区和泉町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
df_tmp = df_receipt.groupby('customer_id')[['amount']].sum()
quant = df_receipt['amount'].quantile(q=[.25,.50,.75])
line25 = quant[0.25]
line50 = quant[0.5]
line75 = quant[0.75]

def quant_judge(x):
  if x < line25:
    return 1
  elif line25 <= x < line50:
    return 2
  elif line50 <= x < line75:
    return 3
  else:
    return 4


df_tmp['quant_cd'] = df_tmp['amount'].apply(quant_judge)
df_tmp.reset_index()

,customer_id,amount,quant_cd
0,CS001113000004,1298,4
1,CS001114000005,626,4
2,CS001115000010,3044,4
3,CS001205000004,1988,4
4,CS001205000006,3337,4
...,...,...,...
8302,CS051513000004,551,4
8303,CS051515000002,265,3
8304,CS052212000002,192,3
8305,CS052514000001,178,3


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer['era'] = df_customer['age'].apply(lambda x : math.floor(x/10)*10 )
df_customer['era'] = df_customer['era'].apply(lambda x: 60 if x >= 60 else x)
df_customer[['customer_id','birth_day','era']]

,customer_id,birth_day,era
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
...,...,...,...
21966,CS002512000474,1959-10-12,50
21967,CS029414000065,1970-10-19,40
21968,CS012403000043,1972-12-16,40
21969,CS033512000184,1964-06-05,50


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer.head()
df_customer[['customer_id','birth_day','era','gender_cd','gender']]

#era列とgender_cd列を文字列型へ変換して結合
df_customer['gender_age_cd'] = df_customer['era'].astype(str) + df_customer['gender_cd'].astype(str)
df_customer[['customer_id','birth_day','era','gender_cd','gender_age_cd','gender']]

,customer_id,birth_day,era,gender_cd,gender_age_cd,gender
0,CS021313000114,1981-04-29,30,1,301,女性
1,CS037613000071,1952-04-01,60,9,609,不明
2,CS031415000172,1976-10-04,40,1,401,女性
3,CS028811000001,1933-03-27,60,1,601,女性
4,CS001215000145,1995-03-29,20,1,201,女性
...,...,...,...,...,...,...
21966,CS002512000474,1959-10-12,50,1,501,女性
21967,CS029414000065,1970-10-19,40,1,401,女性
21968,CS012403000043,1972-12-16,40,0,400,男性
21969,CS033512000184,1964-06-05,50,1,501,女性


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
# get_dummies関数におけるdrop_first=Trueの意味
# drop_first=True → 0,1,9のうち、1,9のみダミー変数化 →　0は全て0の時に合致と解釈
# drop_first=False → 0,1,9のうち、0,1,9全てダミー変数化 →　0に対しても0 or 1を付与

df_dummy = pd.get_dummies(df_customer['gender_cd'], drop_first = True)
pd.concat([df_customer[['customer_id']], df_dummy], axis=1).head(10)

,customer_id,1,9
0,CS021313000114,1,0
1,CS037613000071,0,1
2,CS031415000172,1,0
3,CS028811000001,1,0
4,CS001215000145,1,0
5,CS020401000016,0,0
6,CS015414000103,1,0
7,CS029403000008,0,0
8,CS015804000004,0,0
9,CS033513000180,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]
df_tmp1 = df_tmp.copy()

#sklearnのpreprocessing.StandardScalerクラスよりインスタンスを生成
scaler = preprocessing.StandardScaler()
scaler.fit(df_tmp[['amount']])

df_tmp1['std_amount'] = scaler.transform(df_tmp[['amount']])
df_tmp1[['customer_id','amount','std_amount']].head(10)

,customer_id,amount,std_amount
0,CS006214000001,158,-0.342400
1,CS008415000097,81,-0.502982
2,CS028414000014,170,-0.317374
4,CS025415000050,90,-0.484213
5,CS003515000195,138,-0.384109
6,CS024514000042,30,-0.609342
7,CS040415000178,128,-0.404964
9,CS027514000015,680,0.746226
10,CS025415000134,138,-0.384109
11,CS021515000126,228,-0.196415


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]

df_tmp1 = df_tmp.groupby('customer_id') [['customer_id','amount']].sum().reset_index()
df_tmp1

scaler = preprocessing.StandardScaler()
scaler.fit(df_tmp1[['amount']])
df_tmp1['std_amount'] = scaler.transform(df_tmp1[['amount']])
df_tmp1.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex = True)
df_tmp = df_receipt[cond]
df_tmp1 = df_tmp.groupby('customer_id').sum().reset_index()[['customer_id','amount']]

#常用対数変換時に0.5を足す理由
#"0"に対して対数変換が出来ず、0.5を足す事で離散→連続の近似がより正確になるため。

df_tmp1['log_amount'] = np.log10(df_tmp1['amount'] + 0.5)
df_tmp1.head()

,customer_id,amount,log_amount
0,CS001113000004,1298,3.113442
1,CS001114000005,626,2.796921
2,CS001115000010,3044,3.483516
3,CS001205000004,1988,3.298526
4,CS001205000006,3337,3.523421


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex = True)
df_tmp = df_receipt[cond]
df_tmp1 = df_tmp.groupby('customer_id').sum().reset_index()[['customer_id','amount']]
df_tmp1['log_amount'] = np.log(df_tmp1['amount'] + 0.5)
df_tmp1.head()

,customer_id,amount,log_amount
0,CS001113000004,1298,7.168965
1,CS001114000005,626,6.440149
2,CS001115000010,3044,8.021092
3,CS001205000004,1988,7.595136
4,CS001205000006,3337,8.112977


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
# df_product.drop('profit',axis=1)
df_tmp = df_product.copy()
df_tmp['profit'] = df_tmp['unit_price'] - df_tmp['unit_cost']
df_tmp

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
0,P040101001,4,401,40101,198.0,149.0,49.0
1,P040101002,4,401,40101,218.0,164.0,54.0
2,P040101003,4,401,40101,230.0,173.0,57.0
3,P040101004,4,401,40101,248.0,186.0,62.0
4,P040101005,4,401,40101,268.0,201.0,67.0
...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,70.0
10026,P091503002,9,915,91503,680.0,510.0,170.0
10027,P091503003,9,915,91503,1080.0,810.0,270.0
10028,P091503004,9,915,91503,1130.0,848.0,282.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp[['product_cd','unit_price','unit_cost']]
df_tmp['pct_profit'] = df_tmp[['unit_price','unit_cost']].apply(lambda x : ((x[0] - x[1]) / x[0]), axis=1)
df_tmp.dropna().head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,pct_profit
0,P040101001,4,401,40101,198.0,149.0,0.247475
1,P040101002,4,401,40101,218.0,164.0,0.247706
2,P040101003,4,401,40101,230.0,173.0,0.247826
3,P040101004,4,401,40101,248.0,186.0,0.250000
4,P040101005,4,401,40101,268.0,201.0,0.250000


---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
# (100x - 100) / 100x = 0.3
# 100x - 100 = 30x
# 70x = 100
x = 10/7

df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'] * x
np.floor(df_tmp['new_price'])
df_tmp = pd.concat([df_tmp[['product_cd','unit_cost']],np.floor(df_tmp['new_price'])],axis=1)
df_tmp = df_tmp[['product_cd','new_price','unit_cost']]

df_tmp['pct_profit'] = df_tmp[['new_price','unit_cost']].apply(lambda x : (x[0]-x[1])/x[0] , axis=1)
df_tmp[['product_cd','pct_profit']].dropna()

,product_cd,pct_profit
0,P040101001,0.297170
1,P040101002,0.299145
2,P040101003,0.299595
3,P040101004,0.298113
4,P040101005,0.299652
...,...,...
10025,P091503001,0.300000
10026,P091503002,0.299451
10027,P091503003,0.299914
10028,P091503004,0.299752


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'] * x

#round関数はキーを列名、値を桁数とする辞書を引数に指定すると、各列に個別に桁数を指定できる。
df_tmp.round({'new_price' : 0})

df_tmp['pct_profit'] = df_tmp[['new_price','unit_cost']].apply(lambda x : (x[0]-x[1])/x[0] , axis=1)
df_tmp[['product_cd','pct_profit']].dropna()

,product_cd,pct_profit
0,P040101001,0.3
1,P040101002,0.3
2,P040101003,0.3
3,P040101004,0.3
4,P040101005,0.3
...,...,...
10025,P091503001,0.3
10026,P091503002,0.3
10027,P091503003,0.3
10028,P091503004,0.3


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'] * x

# 切り上げ
np.ceil(df_tmp['new_price'])

df_tmp = pd.concat([df_tmp[['product_cd','unit_cost']],np.ceil(df_tmp['new_price'])],axis=1)
df_tmp = df_tmp[['product_cd','new_price','unit_cost']]

df_tmp['pct_profit'] = df_tmp[['new_price','unit_cost']].apply(lambda x : (x[0]-x[1])/x[0] , axis=1)
df_tmp[['product_cd','pct_profit']].dropna()

,product_cd,pct_profit
0,P040101001,0.300469
1,P040101002,0.302128
2,P040101003,0.302419
3,P040101004,0.300752
4,P040101005,0.302083
...,...,...
10025,P091503001,0.300000
10026,P091503002,0.300412
10027,P091503003,0.300518
10028,P091503004,0.300330


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['unit_price_addtax'] = df_product['unit_price'] * 1.1
np.floor(df_tmp['unit_price_addtax'])
pd.concat( [df_tmp[['product_cd']],np.floor(df_tmp['unit_price_addtax'])], axis=1).head()

,product_cd,unit_price_addtax
0,P040101001,217.0
1,P040101002,239.0
2,P040101003,253.0
3,P040101004,272.0
4,P040101005,294.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [ ]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [ ]:
df_tmp = df_receipt.merge(df_product,how='inner', on="product_cd").groupby('customer_id').sum().reset_index()[['customer_id','amount']]
df_tmp

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337
...,...,...
8302,CS051513000004,551
8303,CS051515000002,265
8304,CS052212000002,192
8305,CS052514000001,178


In [ ]:
df_tmp2 = df_receipt.merge(df_product,how='inner', on="product_cd")
df_tmp2
df_tmp2 = df_tmp2.query('category_major_cd == 7').groupby('customer_id')[['amount']].sum().reset_index()
df_tmp2.columns = ['customer_id', 'amount_category07']
df_tmp2

df_tmp3 = df_tmp.merge(df_tmp2, how="inner", on='customer_id')
pct_series = df_tmp3[['amount','amount_category07']].apply(lambda x : (x[1]/x[0]), axis=1)
df_tmp4 = pd.concat([df_tmp3, pct_series],axis=1)
df_tmp4.columns = ['customer_id', 'amount', 'amount_category07', 'pct']
df_tmp4

,customer_id,amount,amount_category07,pct
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
...,...,...,...,...
6860,CS051212000001,336,68,0.202381
6861,CS051513000004,551,233,0.422868
6862,CS052212000002,192,102,0.531250
6863,CS052514000001,178,68,0.382022


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_receipt.shape

(104681, 9)

In [ ]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,postal_cd_flag,era,gender_age_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0,30,301
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,1,60,609
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,1,40,401
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,0,60,601
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,1,20,201


In [ ]:
df_tmp = df_receipt.drop_duplicates(['sales_ymd','customer_id'])
df_tmp2 = df_customer.merge(df_tmp, how="left", on="customer_id")[['customer_id','sales_ymd','application_date']]
df_tmp3 = df_tmp2.dropna().copy()
df_tmp3['sales_ymd_datetime'] = pd.to_datetime(df_tmp3['sales_ymd'].astype(int).astype(str))
df_tmp3
df_tmp3['application_date_datetime'] = pd.to_datetime(df_tmp3['application_date'].astype(int).astype(str))
df_tmp3['diff_date'] = df_tmp3['sales_ymd_datetime'] - df_tmp3['application_date_datetime']
df_tmp3[['customer_id','sales_ymd_datetime','application_date_datetime','diff_date']]

,customer_id,sales_ymd_datetime,application_date_datetime,diff_date
2,CS031415000172,2017-05-07,2015-05-29,709 days
3,CS031415000172,2017-10-26,2015-05-29,881 days
4,CS031415000172,2019-03-25,2015-05-29,1396 days
5,CS031415000172,2017-01-11,2015-05-29,593 days
6,CS031415000172,2019-01-22,2015-05-29,1334 days
...,...,...,...,...
46068,CS029414000065,2018-12-15,2015-03-13,1373 days
46069,CS029414000065,2018-03-05,2015-03-13,1088 days
46070,CS029414000065,2018-06-11,2015-03-13,1186 days
46071,CS029414000065,2019-10-28,2015-03-13,1690 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]
df_tmp = df_tmp.drop_duplicates(['sales_ymd','customer_id'])
df_tmp2 = df_customer.merge(df_tmp[['customer_id','sales_ymd','amount']], how="inner", on="customer_id")
df_tmp3 = df_tmp2[['customer_id', "sales_ymd",'application_date']]

series_sales_ymd_datetime = pd.to_datetime(df_tmp2['sales_ymd'].astype(str))
series_application_date_datetime = pd.to_datetime(df_tmp2['application_date'].astype(str))

df_tmp4 = pd.concat([df_tmp3,series_sales_ymd_datetime,series_application_date_datetime],axis=1)
df_tmp4.columns = ['customer_id', 'sales_ymd', 'application_date', 'sales_ymd_datetime',
       'application_date_datetime']
df_tmp4['diff_day'] = df_tmp4['sales_ymd_datetime'] - df_tmp4['application_date_datetime']
test = relativedelta(df_tmp4['sales_ymd_datetime'].iloc[0],df_tmp4['application_date_datetime'].iloc[0])
test.months

11

In [ ]:
def use_relativedelta(x):
  return relativedelta(x[0],x[1]).years * 12 + relativedelta(x[0],x[1]).months

series_diff_month = df_tmp4[['sales_ymd_datetime','application_date_datetime']].apply(use_relativedelta, axis=1)
df_tmp5 = pd.concat([df_tmp4,series_diff_month],axis=1)


In [ ]:
df_tmp5.columns = ['customer_id','sales_ymd','application_date','sales_ymd_datetime','application_date_datetime','diff_day','diff_month']
df_tmp5

,customer_id,sales_ymd,application_date,sales_ymd_datetime,application_date_datetime,diff_day,diff_month
0,CS031415000172,20170507,20150529,2017-05-07,2015-05-29,709 days,23
1,CS031415000172,20171026,20150529,2017-10-26,2015-05-29,881 days,28
2,CS031415000172,20190325,20150529,2019-03-25,2015-05-29,1396 days,45
3,CS031415000172,20170111,20150529,2017-01-11,2015-05-29,593 days,19
4,CS031415000172,20190122,20150529,2019-01-22,2015-05-29,1334 days,43
...,...,...,...,...,...,...,...
32406,CS029414000065,20181215,20150313,2018-12-15,2015-03-13,1373 days,45
32407,CS029414000065,20180305,20150313,2018-03-05,2015-03-13,1088 days,35
32408,CS029414000065,20180611,20150313,2018-06-11,2015-03-13,1186 days,38
32409,CS029414000065,20191028,20150313,2019-10-28,2015-03-13,1690 days,55


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]
df_tmp = df_tmp.drop_duplicates(['sales_ymd','customer_id'])
df_tmp2 = df_customer.merge(df_tmp[['customer_id','sales_ymd','amount']], how="inner", on="customer_id")
df_tmp3 = df_tmp2[['customer_id', "sales_ymd",'application_date']]

series_sales_ymd_datetime = pd.to_datetime(df_tmp2['sales_ymd'].astype(str))
series_application_date_datetime = pd.to_datetime(df_tmp2['application_date'].astype(str))

df_tmp4 = pd.concat([df_tmp3,series_sales_ymd_datetime,series_application_date_datetime],axis=1)
df_tmp4.columns = ['customer_id', 'sales_ymd', 'application_date', 'sales_ymd_datetime',
       'application_date_datetime']

def use_relativedelta(x):
  return relativedelta(x[0],x[1]).years

series_diff_year = df_tmp4[['sales_ymd_datetime','application_date_datetime']].apply(use_relativedelta, axis=1)
df_tmp5 = pd.concat([df_tmp4,series_diff_year],axis=1)


df_tmp5.columns = ['customer_id','sales_ymd','application_date','sales_ymd_datetime','application_date_datetime','diff_year']
df_tmp5

,customer_id,sales_ymd,application_date,sales_ymd_datetime,application_date_datetime,diff_year
0,CS031415000172,20170507,20150529,2017-05-07,2015-05-29,1
1,CS031415000172,20171026,20150529,2017-10-26,2015-05-29,2
2,CS031415000172,20190325,20150529,2019-03-25,2015-05-29,3
3,CS031415000172,20170111,20150529,2017-01-11,2015-05-29,1
4,CS031415000172,20190122,20150529,2019-01-22,2015-05-29,3
...,...,...,...,...,...,...
32406,CS029414000065,20181215,20150313,2018-12-15,2015-03-13,3
32407,CS029414000065,20180305,20150313,2018-03-05,2015-03-13,2
32408,CS029414000065,20180611,20150313,2018-06-11,2015-03-13,3
32409,CS029414000065,20191028,20150313,2019-10-28,2015-03-13,4


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]
df_tmp = df_tmp.drop_duplicates(['sales_ymd','customer_id'])
df_tmp2 = df_customer.merge(df_tmp[['customer_id','sales_ymd','amount']], how="inner", on="customer_id")
df_tmp3 = df_tmp2[['customer_id', "sales_ymd",'application_date']]

series_sales_ymd_datetime = pd.to_datetime(df_tmp2['sales_ymd'].astype(str))
series_application_date_datetime = pd.to_datetime(df_tmp2['application_date'].astype(str))

In [ ]:
series_sales_ymd_timestamp = series_sales_ymd_datetime.apply(lambda x : x.timestamp())

series_application_date_datetime = pd.to_datetime(df_tmp2['application_date'].astype(str))
series_application_date_timestamp = series_application_date_datetime.apply(lambda x : x.timestamp())
series_timestamp_diff = series_sales_ymd_timestamp - series_application_date_timestamp
series_timestamp_diff

pd.concat([df_tmp3,series_timestamp_diff],axis=1)

,customer_id,sales_ymd,application_date,0
0,CS031415000172,20170507,20150529,61257600.0
1,CS031415000172,20171026,20150529,76118400.0
2,CS031415000172,20190325,20150529,120614400.0
3,CS031415000172,20170111,20150529,51235200.0
4,CS031415000172,20190122,20150529,115257600.0
...,...,...,...,...
32406,CS029414000065,20181215,20150313,118627200.0
32407,CS029414000065,20180305,20150313,94003200.0
32408,CS029414000065,20180611,20150313,102470400.0
32409,CS029414000065,20191028,20150313,146016000.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
series_tmp = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
series_tmp = series_tmp.apply(lambda x : x.weekday())
df_tmp = df_receipt.copy()
df_tmp2 = pd.concat([df_tmp[['customer_id','sales_ymd']],series_tmp],axis=1)
df_tmp2.columns = ['customer_id', 'sales_ymd', 'erased_day']
df_tmp2
series_tmp = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
series_tmp2 =  series_tmp.apply(lambda x : x - relativedelta(days = x.weekday()))
series_tmp2

0        2018-10-29
1        2018-11-12
2        2017-07-10
3        2019-02-04
4        2018-08-20
            ...    
104676   2018-02-19
104677   2019-09-09
104678   2017-03-06
104679   2017-03-27
104680   2019-04-22
Name: sales_ymd, Length: 104681, dtype: datetime64[ns]

In [ ]:
# series_test =  series_tmp.apply(lambda x : x - x.weekday())

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(frac=0.01).head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,postal_cd_flag,era,gender_age_cd
18283,CS002715000239,山内 美智子,9,不明,1947-04-12,71,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20170703,0-00000000-0,1,60,609
7349,CS007502000047,鹿手袋 サダヲ,0,男性,1965-07-25,53,276-0022,千葉県八千代市上高野**********,S12007,20151003,0-00000000-0,0,50,500
14328,CS040802000004,吉田 サダヲ,0,男性,1937-01-26,82,226-0015,神奈川県横浜市緑区三保町**********,S14040,20150728,0-00000000-0,0,60,600
12162,CS021212000035,藤山 夏希,1,女性,1991-01-09,28,254-0004,神奈川県平塚市小鍋島**********,S14021,20151008,0-00000000-0,0,20,201
21591,CS026515000031,浜口 なぎさ,1,女性,1960-11-03,58,251-0047,神奈川県藤沢市辻堂**********,S14026,20150512,E-20100826-F,0,50,501


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
_, df_tmp = train_test_split(df_customer, test_size=0.1, train_size=0.9, stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd')[['customer_id']].count().reset_index()

,gender_cd,customer_id
0,0,298
1,1,1793
2,9,107


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]

df_tmp2 = df_tmp.groupby('customer_id')[["amount"]].sum().reset_index()
df_tmp2['amount']
scaler = preprocessing.StandardScaler()
scaler.fit(df_tmp2[['amount']])

StandardScaler()

In [ ]:
array_std_amount = scaler.transform(df_tmp2[['amount']])
df_tmp2['std_amount'] = array_std_amount
df_tmp2.query('std_amount >= 3 | std_amount < =-3')

,customer_id,amount,std_amount
332,CS001605000009,18925,6.019921
1755,CS006415000147,12723,3.740202
1817,CS006515000023,18372,5.816651
1833,CS006515000125,12575,3.685800
1841,CS006515000209,11373,3.243972
...,...,...,...
7986,CS040214000008,13523,4.034264
8041,CS040414000077,11282,3.210522
8048,CS040415000035,13119,3.885763
8065,CS040415000200,11137,3.157223


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
cond = df_receipt['customer_id'].str.contains('^[^Z]',regex=True)
df_tmp = df_receipt[cond]

df_tmp2 = df_tmp.groupby('customer_id')[["amount"]].sum().reset_index()
df_tmp2['amount']

df_tmp3 = df_tmp2['amount'].quantile([.25, .5, .75])
iqr = df_tmp3[0.75] - df_tmp3[0.25]
lower = df_tmp3[0.25] - 1.5 * iqr
upper = df_tmp3[0.75] + 1.5 * iqr

df_tmp2.query('amount >= @upper | amount <= @lower').sort_index()

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
...,...,...
8068,CS040415000220,10158
8126,CS040515000089,9240
8135,CS040515000225,14867
8171,CS041411000001,9694


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product_1 = df_product.copy()
df_product_1.dropna(inplace=True)
df_product_1

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0
...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0
10026,P091503002,9,915,91503,680.0,510.0
10027,P091503003,9,915,91503,1080.0,810.0
10028,P091503004,9,915,91503,1130.0,848.0


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product_2 = df_product.copy()
df_product_2.fillna( value= {'unit_price':np.round(df_product_2['unit_price'].mean())}, inplace=False)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0
...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0
10026,P091503002,9,915,91503,680.0,510.0
10027,P091503003,9,915,91503,1080.0,810.0
10028,P091503004,9,915,91503,1130.0,848.0


---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product_2 = df_product.copy()
df_product_22 = df_product_2.fillna( value= {'unit_price':np.round(df_product_2['unit_price'].median())}, inplace=False)
df_product_22.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             7
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_tmp = df_product.groupby('category_small_cd')[['unit_price','unit_cost']].median().reset_index()
df_tmp.columns = ['category_small_cd', 'unit_price_median', 'unit_cost_median']
df_product_4 = df_product.copy()
df_product_4 = df_product_4.merge(df_tmp, how="inner", on="category_small_cd")
df_product_4
# # df_product_4[df_product_4['unit_price'].isnull()]


def fillna_median(x):
  if np.isnan(x[0]) == True:
    return round(x[1])
  else:
    return round(x[0])


df_product_4['filled_unit_price'] = df_product_4[['unit_price','unit_price_median']].apply(fillna_median, axis=1)
df_product_4['filled_unit_cost'] = df_product_4[['unit_cost','unit_cost_median']].apply(fillna_median, axis=1)
df_product_4[df_product_4['unit_price'].isnull()]


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_median,unit_cost_median,filled_unit_price,filled_unit_cost
159,P040802007,4,408,40802,NaN,NaN,313.0,235.0,313,235
196,P050103021,5,501,50103,NaN,NaN,132.5,99.5,132,100
496,P050405009,5,504,50405,NaN,NaN,178.0,133.5,178,134
1531,P060802026,6,608,60802,NaN,NaN,270.0,200.0,270,200
2012,P070202092,7,702,70202,NaN,NaN,238.0,179.0,238,179
6296,P080504027,8,805,80504,NaN,NaN,258.0,195.5,258,196
7075,P090204185,9,902,90204,NaN,NaN,694.0,521.0,694,521


---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
df_tmp = df_customer.merge(df_receipt[['customer_id','amount']], how="left", on="customer_id")
df_tmp.fillna(value = 0, inplace=True)
df_tmp1 = df_tmp.groupby('customer_id')[['amount']].sum().reset_index()
amount_all = df_tmp1['amount'].sum()
df_tmp1['pct_amount'] = df_tmp1['amount'] / amount_all
df_tmp1.sort_values('amount', ascending=False)

,customer_id,amount,pct_amount
11297,CS017415000097,23086.0,0.001091
10371,CS015415000185,20153.0,0.000952
17609,CS031414000051,19202.0,0.000907
16223,CS028415000007,19127.0,0.000904
1039,CS001605000009,18925.0,0.000894
...,...,...,...
8346,CS010715000045,0.0,0.000000
8347,CS010715000051,0.0,0.000000
8348,CS010715000054,0.0,0.000000
8349,CS010715000065,0.0,0.000000


---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
df_geocode_1 = df_geocode.groupby('postal_cd')[['longitude','latitude']].mean().reset_index()
df_customer_1 = pd.merge(df_customer, df_geocode_1, how='inner', on="postal_cd")
df_customer_1

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,postal_cd_flag,era,gender_age_cd,longitude,latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0,30,301,139.31779,35.41358
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,0-00000000-0,0,30,300,139.31779,35.41358
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,0-00000000-0,0,30,300,139.31779,35.41358
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,0-00000000-0,0,30,301,139.31779,35.41358
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,0-00000000-0,0,30,301,139.31779,35.41358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21966,CS021612000057,星 朝陽,1,女性,1950-05-07,68,259-1134,神奈川県伊勢原市八幡台**********,S14021,20150405,4-20100829-7,0,60,601,139.30992,35.38544
21967,CS035401000016,白木 蒼甫,0,男性,1971-05-06,47,155-0033,東京都世田谷区代田**********,S13035,20150527,0-00000000-0,1,40,400,139.66281,35.65271
21968,CS003612000043,竹内 那奈,1,女性,1952-07-12,66,182-0033,東京都調布市富士見町**********,S13003,20150608,0-00000000-0,1,60,601,139.53539,35.66020
21969,CS007612000095,会田 陽子,9,不明,1950-12-08,68,276-0043,千葉県八千代市萱田**********,S12007,20150921,4-20100704-8,0,60,609,140.10959,35.73419


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
df_customer_1 = df_customer_1.merge(df_store, how="inner", left_on="application_store_cd", right_on="store_cd")

def distance(list):
  latitude_store = list[0]
  longitude_store = list[1]
  latitude_customer = list[2]
  longitude_customer = list[3]

  l = 6371* math.acos(math.sin(math.radians(latitude_store))
       * math.sin(math.radians(latitude_customer))
       + math.cos(latitude_store)*math.cos(latitude_customer)
       * math.cos(math.radians(longitude_store) - math.radians(longitude_customer))
  )
  return l

df_customer_1['distance'] = df_customer_1[['latitude_y', 'longitude_y', 'latitude_x', 'longitude_x']].apply(distance, axis=1)
df_customer_1[['customer_id', 'distance']].head(10)

,customer_id,distance
0,CS021313000114,4397.250307
1,CS021303000023,4397.250307
2,CS021303000007,4397.250307
3,CS021313000183,4397.250307
4,CS021314000098,4397.250307
5,CS021314000093,4397.250307
6,CS021413000049,4397.250307
7,CS021313000025,4317.706878
8,CS021413000022,4317.706878
9,CS021413000094,4317.706878


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df_receipt_tmp = df_receipt.groupby('customer_id') \
                    .agg(sum_amount=('amount','sum')).reset_index()

df_customer_u = pd.merge(df_customer, df_receipt_tmp,
                         how='left',
                         on='customer_id')

df_customer_u['sum_amount'] = df_customer_u['sum_amount'].fillna(0)

df_customer_u = df_customer_u.sort_values(['sum_amount', 'customer_id'],
                                          ascending=[False, True])

df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'],
                              keep='first', inplace=True)

print('df_customer_cnt:', len(df_customer),
      'df_customer_u_cnt:', len(df_customer_u),
      'diff:', len(df_customer) - len(df_customer_u))

df_customer_cnt: 21971 df_customer_u_cnt: 21941 diff: 30


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_sales_customer = df_receipt.groupby('customer_id') \
                                    .agg({'amount':sum}).reset_index()

df_sales_customer = df_sales_customer.query('amount > 0')

df_tmp = pd.merge(df_customer, df_sales_customer['customer_id'],
                  how='inner', on='customer_id')

df_train, df_test = train_test_split(df_tmp, test_size=0.2, random_state=71)

print('学習データ割合: ', len(df_train) / len(df_tmp))
print('テストデータ割合: ', len(df_test) / len(df_tmp))

学習データ割合:  0.7999036840837949
テストデータ割合:  0.20009631591620516


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [ ]:
# コード例1（自作関数）
df_ts_amount = df_receipt[['sales_ymd', 'amount']].copy()

df_ts_amount['sales_ym'] = df_ts_amount['sales_ymd'].astype('str').str[0:6]

df_ts_amount = df_ts_amount.groupby('sales_ym') \
                                        .agg({'amount':'sum'}).reset_index()

# 長期間データに対する多数のデータセットもループなどで処理できるように関数化
def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start:test_start], df[test_start:test_start + test_size]

df_train_1, df_test_1 = split_data(df_ts_amount, train_size=12,
                                   test_size=6, slide_window=6, start_point=0)

df_train_2, df_test_2 = split_data(df_ts_amount, train_size=12,
                                   test_size=6, slide_window=6, start_point=1)

df_train_3, df_test_3 = split_data(df_ts_amount, train_size=12,
                                   test_size=6, slide_window=6, start_point=2)

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')

df_tmp['is_buy_flag'] = np.where(df_tmp['amount'].isnull(), 0, 1)

rs = RandomUnderSampler(random_state=71)

df_down_sampling, _ = rs.fit_resample(df_tmp, df_tmp.is_buy_flag)

print('0の件数', len(df_down_sampling.query('is_buy_flag == 0')))
print('1の件数', len(df_down_sampling.query('is_buy_flag == 1')))

0の件数 8306
1の件数 8306


---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [ ]:
df_gender_std = df_customer[['gender_cd', 'gender']].drop_duplicates()

df_customer_std = df_customer.drop(columns='gender')

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！